<a href="https://colab.research.google.com/github/yunjeooong/MiraeassetFin/blob/main/%E1%84%8C%E1%85%A5%E1%86%AB%E1%84%8E%E1%85%A5%E1%84%85%E1%85%B5_%E1%84%8F%E1%85%B3%E1%84%85%E1%85%A9%E1%86%AF%E1%84%85%E1%85%B5%E1%86%BC_%E1%84%8C%E1%85%AE%E1%86%BC%E1%84%87%E1%85%A9%E1%86%A8%E1%84%82%E1%85%B2%E1%84%89%E1%85%B3%E1%84%8C%E1%85%A6%E1%84%80%E1%85%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버증권 뉴스 크롤링

---



# 필요한 라이브러리 임포트

데이터 분석과 시각화를 위해 필요한 라이브러리와 정규 표현식 모듈을 임포트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 정규식을 사용하는 라이브러리
import re

웹 브라우저 자동화: Selenium 라이브러리 설치

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


단어 구름(워드 클라우드)을 생성: WordCloud 라이브러리 설치

In [ ]:
!pip install wordcloud

엑셀 파일 처리: openpyxl 라이브러리 설치

In [ ]:
!pip install openpyxl

드라이버 자동 관리: webdriver_manager 라이브러리 설치

In [ ]:
!pip install webdriver_manager

# 데이터 분석, 시각화, 워드 클라우드 생성, 웹 자동화를 위한 라이브러리와 모듈을 임포트

*   데이터 분석 및 조작
*   데이터 시각화
*   워드 클라우드 생성
*   시간 관련 기능
*   웹 브라우저 자동화
*   요소 선택
*   Chrome 드라이버 자동 관리




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 웹 드라이버 설정 및 실행

헤드리스 모드 및 기타 설정을 적용한 Chrome 웹 드라이버를 생성하여 브라우저를 실행

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--single-process')

browser = webdriver.Chrome(options=chrome_options)

# 페이지 URL 리스트 생성
네이버 금융 뉴스의 1페이지부터 5페이지까지의 URL 리스트를 생성

In [ ]:
# 모든 페이지 URL 리스트
base_url = 'https://finance.naver.com/news/news_list.naver?mode=RANK&page='
page_urls = [base_url + str(page) for page in range(1, 5)]  # 1~ 5 페이지

# 뉴스 제목과 URL 크롤링 및 본문과 날짜/시간 추출

페이지 URL을 통해 뉴스 제목과 URL을 크롤링한 후, 각 뉴스 기사에서 본문과 날짜/시간 정보를 추출

In [ ]:
news_data = []

for page_url in page_urls:
    browser.get(page_url)
    time.sleep(3)  # 페이지 로딩을 기다리기 위해 잠시 대기
    news_title_elements = browser.find_element(By.CLASS_NAME, "hotNewsList")
    news_links = news_title_elements.find_elements(By.TAG_NAME, "a")

    for link in news_links:
        title = link.text
        news_url = link.get_attribute('href')
        news_data.append((title, news_url))

# 뉴스 본문, 날짜, 시간 크롤링
news_content_list = []

for title, news_url in news_data:
    browser.get(news_url)
    time.sleep(3)
    try:
        content_element = browser.find_element(By.CSS_SELECTOR, '#newsct_article .go_trans._article_content')
        content = content_element.text

        # 날짜와 시간 크롤링 (예시: 날짜와 시간이 특정 CSS 셀렉터에 있다고 가정)
        date_time_element = browser.find_element(By.CSS_SELECTOR,
          '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')
        date_time = date_time_element.text

    except Exception as e:
        content = "본문을 가져올 수 없습니다."

    news_content_list.append((title, content, date_time))


날짜와 시간 관련 작업을 위해 datetime 모듈을 임포트

In [ ]:
from datetime import datetime

# 뉴스 데이터 데이터프레임 변환 및 엑셀 파일로 저장

뉴스 데이터를 데이터프레임으로 변환한 후, 현재 날짜를 포함한 파일명으로 엑셀 파일로 저장하고, 웹 드라이버를 종료

In [ ]:
# 데이터프레임으로 변환
df = pd.DataFrame(news_content_list, columns=["뉴스제목", "본문", "날짜"])

# 엑셀 파일로 저장
current_date = datetime.now().strftime("%Y%m%d")
file_name = f'naver_news_content_{current_date}.xlsx'
df.to_excel(file_name, index=False)

# 드라이버 종료
browser.quit()

뉴스 데이터를 엑셀 파일로 저장한 후, 해당 파일을 읽어 데이터프레임으로 재로드하여 내용을 확인

In [ ]:
# 엑셀 파일로 저장
current_date = datetime.now().strftime("%Y%m%d")
file_name = f'naver_news_content_{current_date}.xlsx'
df.to_excel(file_name, index=False)

df = pd.read_excel(file_name)
df

,뉴스제목,본문,날짜
0,‘5800만→58만→170만원’ 탁재훈 흑역사 쓴 비트코인…지금..,[유튜브 채널 '시즌비시즌' 캡처]\n\n\n[헤럴드경제=신동윤 기자] 가상자산 대...,2024.07.30. 오전 8:16
1,[단독] 총수 구속 코앞두고 카카오 임원 주식 처분 ‘시끌’,커뮤니티서 갑론을박\n논란 일자 “금지 기간 약 일주일 전 체결” 해명\n\n카카오...,2024.07.30. 오후 6:31
2,"'3조' 수주 잭팟 터지나…상승 랠리 탄 '이 종목', 52주 ..","2Q 어닝 서프라이즈 한국항공우주, 하반기 수주 기대감↑\n육군이 국산 기동헬기인 ...",2024.07.30. 오후 4:03
3,"""비트코인 팔지 말라더니""…美정부, BTC 20억달러 이전?","비트코인 이날 오전 한때 6만5000달러까지 하락\n\n美정부, 비트코인 3만여개 ...",2024.07.30. 오후 6:45
4,"외국인은 벌써 사 모았네…이달 30% 뛴 이 주식, ""더 오른다..",[오늘의 포인트]\n/사진=임종철 기자.\n\n올해 2분기 호실적을 발표한 삼성바이...,2024.07.30. 오전 11:40
...,...,...,...
95,"이복현, 큐텐에 ""양치기 소년 행태""…구영배 ""800억 있지만 ..","국회 정무위 긴급 현안 질의\n금감원 ""자금 추적 과정서 강한 불법 흔적 찾아""\n...",2024.07.30. 오후 6:10
96,‘아무나 만든’ 상품권 해피머니 禍 키웠다,티메프 판매 상품권 제휴사 결제거절\n머지포인트 사태 이전인 2021년\n관련 규제...,2024.07.30. 오전 11:35
97,오늘의 증시일정 (7월31일),◇신주배정기준일=뉴보텍(유상)\n\n◇신규상장=피앤에스미캐닉스\n\n◇추가상장=미래...,2024.07.30. 오후 5:56
98,확신의 '슈퍼사이클'...조선3사 주가는 '엇갈림' [마켓인사이..,"미국과 일본, 영국 등 주요국 통화정책회의, 주요 빅테크들의 실적 발표 등 여러 이...",2024.07.30. 오후 5:18


# 데이터 텍스트 전처리

## 패키지 설치 및 전처리 함수 정의

### 패키지 설치
자연어 처리 라이브러리 설치 및 임포트

한국어 문장 분리 라이브러리 설치

In [ ]:
!pip install kss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

한국어 문장 분리 모듈 임포트

In [ ]:
import kss

AttributeError: module 'xlrd' has no attribute 'xlsx'

한국어 자연어 처리 라이브러리 설치

In [ ]:
# soynlp: 한국어 텍스트 데이터를 분석하고 처리하는 라이브러리
!pip install soynlp

한국어 형태소 분석기 설치

In [ ]:
!pip install konlpy

MeCab 형태소 분석기 설치

In [ ]:
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Keras 텍스트 전처리 모듈 임포트

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

### 불용어 로드 및 제거 함수 정의

불용어를 로드하고, 주어진 텍스트에서 불용어를 제거하는 함수를 정의

In [ ]:
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        stopwords = {line.strip() for line in file if line.strip()}
    return stopwords

# 파일 경로
stopwords_filepath = '/content/drive/MyDrive/Colab Notebooks/stopwords-ko.txt'
stopwords = load_stopwords(stopwords_filepath)

# 불용어 제거 함수
def delete_stopwords(context):
    preprocessed_text = []
    for text in context:
        text = [w for w in text.split(' ') if w not in stopwords] # text를 띄어쓰기로 구분하고 불용어사전에 없는 데이터만 리스트에 저장
        preprocessed_text.append(' '.join(text)) # 띄어쓰기를 적용하여 리스트에 추가
    return preprocessed_text


### HTML 태그 제거 함수 정의

문자열에서 HTML 태그를 제거하는 함수를 정의

In [ ]:
# HTML 태그를 제거하는 함수
# re.sub(r'정규식', '대체될 텍스트', '문자열'): 문자열에서 패턴을 검색하고, 해당 패턴을 다른 문자열로 대체하는 역할
def delete_html_tag(context):
    processed_context = []
    for text in context:
        text = re.sub(r'<[%>]+Ws+(?=<)|<[^>]+>', '', text).strip()  # text에서 정규식에 해당하는 조건이 True가 되면 ''(제거)
        if text:
          processed_context.append(text)

    return processed_context

문장 분리 함수 정의

In [ ]:
# 문장을 분리하기 위한 함수
def sentence_separator(processed_context):
  splited_context = []

  for text in processed_context:
    text = text.strip()  # 양쪽 공백 제거
    if text:
      splited_text = kss.split_sentences(text)   # 문장 단위로 분리
      splited_context.extend(splited_text) # 분리된 문장을 리스트에 추가
  return splited_context

### 이메일 주소 제거 함수 정의

텍스트에서 이메일 주소를 제거하는 함수를 정의

In [ ]:
# 정규식을 통해 이메일 제거하는 함수
def delete_email(context):
    preprocessed_text = []
    for text in context:
        text = re.sub(r'[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

### 해시태그 제거 함수 정의

텍스트에서 해시태그를 제거하는 함수를 정의

In [ ]:
# 정규식을 통한 해시태그 제거
def delete_hashtag(context):
  preprocessed_text = []
  for text in context:
    text = re.sub(r'#\S+', '', text).strip()
    if text:
      preprocessed_text.append(text)
  return preprocessed_text

### 저작권 관련 텍스트 제거 함수 정의

텍스트에서 저작권 관련 정보(저작권자, ©, (C) 등)를 제거하는 함수를 정의

In [ ]:
# 정규식을 통한 링크 제거
def delete_copyright(context):
    re_patterns = [
        r'\<저작권자(\(c\)|©|(C)|(\(C\))).+?\>',
        r'(Copyrights)|(\(c\))|(\(C\))|©|(C)|'
    ]
    preprocessed_text = []
    for text in context:
        for re_pattern in re_patterns:  # 위 패턴들만큼 반복
            text = re.sub(re_pattern, "", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

### 특수 문자 제거 함수 정의

텍스트에서 특수 문자를 제거하고, 한글, 영문, 숫자, 공백만 남기는 함수를 정의

In [ ]:
def delete_special_characters(context):
    preprocessed_text = []
    for text in context:
        text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

SoyNLP의 텍스트 정규화 모듈 임포트

In [ ]:
from soynlp.normalizer import *

### 텍스트 공백 정규화 함수 정의

텍스트의 공백을 정규화하여 일관된 형식으로 변환하는 함수를 정의

In [ ]:
def normalize_spacing(context):
    normalized_text = []
    for text in context:
        text = normalize(text)
        normalized_text.append(text)
    return normalized_text

### 형태소 어간 추출 함수 정의

Mecab을 사용하여 텍스트에서 형태소를 분석하고 어간을 추출하는 함수를 정의

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

def extract_stems(context):
    stemmed_text = []
    for text in context:
        stemmed_sentence = []
        for word in mecab.morphs(text):
            stemmed_sentence.append(word[:-1])  # 어간 추출
        stemmed_text.append(' '.join(stemmed_sentence))
    return stemmed_text

### 중복 문장 제거 함수 정의

문장 리스트에서 중복된 문장을 제거하고, 원래의 순서를 유지하는 함수를 정의

In [ ]:
# OrderedDict: 순서가 있는 딕셔너리(Dictionary) 타입
# 기존의 딕셔너리와는 달리 아이템이 추가된 순서를 기억
from collections import OrderedDict

# 중복된 문장을 제거하는 함수
def duplicated_sentence_normalizer(context):
  context = list(OrderedDict.fromkeys(context))
  return context

### 형태소 필터링 함수 정의

형태소 분석을 통해 명사, 동사, 형용사가 모두 포함된 문장만을 필터링하여 반환하는 함수를 정의

In [ ]:
# 객체 생성
mecab = Mecab()
morphs = mecab.pos('아버지가방에들어가신다', join=False)

# 형태소 분석 필터링
# 명사(NN), 동사(V), 형용사(J)의 포함 여부에 따라 문장 필터링
# 주어진 텍스트에서 명사, 동사, 형용사가 모두 포함된 텍스트들만을 추출하여 반환
def morph_filter(context):
  NN_TAGS = ['NNG', 'NNP', 'NNB', 'NP']
  V_TAGS = ['VV', 'VA', 'VX', 'VCP', 'VCN', 'XSN', 'XSA', 'XSV']
  J_TAGS = ['JKS', 'J', 'JO', 'JK', 'JKC', 'JKG', 'JKB', 'JKV', 'JKQ',
              'JX', 'JC', 'JKI', 'JKO', 'JKM', 'ETM']
  preprocessed_text = []
  for text in context:
      morphs = mecab.pos(text, join=False)

      # 존재하는지에 대한 여부
      nn_flag = False
      v_flag = False
      j_flag = False
      for morph in morphs:
        pos_tags = morph[1].split("+")   # ('신다', 'EP+EC') -> ['EP', 'EC']
        for pos_tag in pos_tags:
          if not nn_flag and pos_tag in NN_TAGS:  # True면 False
              nn_flag = True
          if not v_flag and pos_tag in V_TAGS:
              v_flag = True
          if not j_flag and pos_tag in J_TAGS:
              j_flag = True
        if nn_flag and v_flag and j_flag:   # 모두 True라면
          preprocessed_text.append(text)
          break
  return preprocessed_text

### 텍스트 길이 필터링 함수 정의

주어진 최소 및 최대 길이 범위 내에 있는 문장만을 필터링하여 반환하는 함수를 정의

In [ ]:
def min_max_filter(min_len, max_len, context):
      filtered_text = []

      for text in context:
        if min_len <= len(text) and len(text) < max_len:
          filtered_text.append(text)

      return filtered_text

## 뉴스 기사 전처리 및 텍스트 결합

엑셀 파일에서 뉴스 제목과 본문을 읽어와서 다양한 전처리 작업을 수행한 후, 전처리된 텍스트를 리스트에 저장

In [ ]:
# 전처리된 텍스트와 인덱스를 저장할 리스트 초기화
preprocessed_texts = []
indices = []

for index, row in df.iterrows():
    title = row['뉴스제목']  # 엑셀 파일의 '제목' 컬럼
    content = row['본문']   # 엑셀 파일의 '본문' 컬럼

    # article 객체를 가정하고 title과 text에 할당
    article = type('', (), {})()  # 빈 객체 생성
    article.title = title
    article.text = content

    print('title:', article.title)
    print('content:', article.text)
    print('-' * 80)
    context = article.text.split('\n')

    processed_context = delete_stopwords(context)
    processed_context = delete_html_tag(processed_context)
    splited_context = sentence_separator(processed_context)
    processed_context = delete_email(splited_context)
    processed_context = delete_hashtag(processed_context)
    processed_context = delete_copyright(processed_context)
    processed_context = delete_special_characters(processed_context)
    processed_context = normalize_spacing(processed_context)
    processed_context = extract_stems(processed_context)
    normalized_context = duplicated_sentence_normalizer(processed_context)
    post_processed_context = morph_filter(normalized_context)
    post_processed_context = min_max_filter(20, 60, post_processed_context)

    # 기사별로 전처리된 텍스트를 하나의 문자열로 결합하여 리스트에 추가
    if post_processed_context:
        combined_text = ' '.join(post_processed_context)
        preprocessed_texts.append(combined_text)
        indices.append(index)

NameError: name 'df' is not defined

## 전처리된 텍스트 내용 확인

전처리된 텍스트의 문서 수를 출력하고, 상위 5개의 문서 내용을 확인하여 출력

In [ ]:
# preprocessed_texts 내용 확인
print(f"Number of documents: {len(preprocessed_texts)}")
for i in range(min(5, len(preprocessed_texts))):
    print(f"Document {i+1}:")
    print(preprocessed_texts[i])
    print("-" * 80)

Number of documents: 98
Document 1:
일본 증시 시세판 앞 지나가는 남성 사진  AP 연합뉴스 상장폐지라는 페널티가 있어야 기업들이 주가부양을 위해 노력할 것이란 논리였다 한국개인투자자연합회는 자진상장폐지 권유 종목 39종을 발표하기도 했다 최근 장부가치를 밑도는 상장사에 대한 공개매수를 통한 상장폐지가 잇따르면서 개인투자자들은 피해를 보게 됐다 기업가치보다 현저히 낮은 가격에 공개매수가격이 정해지면서 이상으로 주가가 오르지 않기 때문이다 공개매수가보다 낮은 가격에 주식을 매수했던 투자자들은 시장에 매물을 던지게 된다 공개매수 가격이 주당 순자산3136원보다도 낮아 비판을 받았다 신성통상은 배당에 인색했고 주가가 다시 반토막 난 바 있다 한 업계 관계자는 주주 환원에 인색한 창업주 일가 행태를 전형적으로 보여준 사례라고 비판했다 2012년 SK ES의 자회사였던 코원에너지서비스는 065배의 PBR에 공개매수상장폐지가 진행됐다 배당금은 상폐 전 77억5600억원에서 상폐 후엔 2600억원으로 급증했다 최근 증권가와 정치권에서 비판이 높아지고 있는 두산그룹 사업구조 개편도 논란이다
--------------------------------------------------------------------------------
Document 2:
이제는 코리안 드림Korean Dream이 아닌 마이 드림My Dream 기간 가족에게 송금하는 비율은 절반 이하45로 떨어졌다 현재까지 해외송금 서비스를 이용한 국내 거주 외국인 근로자 대부분은 급여소득자로 전체 81를 차지했다 국내 거주 외국인 근로자 소비도 대부분 2030세대가 이끌었다 올해 상반기 2030세대가 카드 이용건수와 금액에서 64와 57를 차지했다 특히 전남은 전국 평균 대비 카드 이용건수12와 이용금액7 10p 높았다 먼저 전남 지역은 영암군에 1만여명의 외국인이 거주하는 소비도 최고를 기록했다 최고 매출 상승 업종 역시 유통으로 33를 차지했다 영암군은 대불국가산업단지가 위치

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 중복 뉴스 확인

## TF-IDF 계산 및 단어 인덱스
처리된 텍스트 데이터를 기반으로 TF-IDF 값을 계산하고, 단어 사전에서 단어와 인덱스를 출력

In [ ]:
# TF-IDF 값 계산
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preprocessed_texts)

# 단어사전에서 단어와 idx값 출력
for key, idx in sorted(vectorizer.vocabulary_.items()):
    print(f'{key}: {idx}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
kpx그룹이: 773
kpx홀딩스: 774
krx: 775
ktop에서: 776
k방산: 777
lg생활건강은: 778
lg에너지솔루션373220도: 779
lg에너지솔루션492: 780
lg에너지솔루션과: 781
lg에너지솔루션와: 782
lg에너지솔루션은: 783
lg에너지솔루션이: 784
lg엔솔gm: 785
lpr을: 786
ls머트리얼즈608: 787
ls일렉트릭: 788
ls일렉트릭은: 789
m7: 790
m7매그니피센트7: 791
ms에: 792
mz세대: 793
nda비밀유지협약를: 794
nh농협이: 795
nh투자증권: 796
option을: 797
pbr: 798
pbr에: 799
pbr은: 800
pbr을: 801
pce: 802
pf: 803
plus플러스와: 804
posco홀딩스229: 805
posco홀딩스가: 806
pp: 807
pp사업부의: 808
rd: 809
rise: 810
rise라이즈로: 811
sg1324: 812
si서: 813
sk: 814
skc는: 815
sk오션플랜트100090214: 816
sk하이닉스: 817
sk하이닉스000660215: 818
sk하이닉스000660가: 819
sk하이닉스215: 820
sk하이닉스215가: 821
sk하이닉스가: 822
sk하이닉스는: 823
sk하이닉스도: 824
sk하이닉스를: 825
sk하이닉스에: 826
sk하이닉스와: 827
sk하이닉스의: 828
sns: 829
sns에: 830
sol: 831
sol로: 832
sp500: 833
stx중공업889과: 834
tiger: 835
top: 836
tsmc: 837
tsmc밸류체인factset은: 838
vix: 839
가격: 840
가격도: 841
가격에: 842
가격으로: 843
가격은: 844
가격이: 845
가격이다: 846
가계대출: 847
가계대출은: 848
가까운: 849
가까울수록: 850
가까이: 851
가깝게: 852
가나: 

## TF-IDF 벡터 정규화

TF-IDF 변환기를 사용하여 문서들을 TF-IDF 벡터로 변환하고, 이를 정규화하여 출력

In [ ]:
# TF-IDF 변환기인 tfidfv를 사용하여 주어진 문서들을 TF-IDF 벡터로 변환 후 array형태로 변환
sk_tf_idf = vectorizer.transform(preprocessed_texts).toarray()

# TF-IDF 벡터 정규화 (L2 정규화)
from sklearn.preprocessing import normalize
normalized_tf_idf = normalize(sk_tf_idf, norm='l2')
print(normalized_tf_idf)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## K-means 군집화 및 클러스터 결과 확인

문서들을 K-means 알고리즘으로 군집화하고, 각 문서의 클러스터 레이블을 데이터프레임에 추가한 후, 클러스터 결과를 출력

In [ ]:
# K-means 군집화
from sklearn.cluster import KMeans

# 문서 수
n = len(preprocessed_texts)

# k 값 계산 (루트(n/2) / 2)
k = int(np.sqrt(n / 2) / 2)

# K-means 군집화
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans.fit(normalized_tf_idf)

df.loc[indices, 'cluster'] = kmeans.labels_

# 클러스터 결과 확인
print("클러스터 결과:")
print(df[['뉴스제목', 'cluster']])

# 각 클러스터에 속하는 기사 출력
for cluster_num in range(k):
    print(f"\n클러스터 {cluster_num}에 속하는 기사들:")
    cluster_articles = df[df['cluster'] == cluster_num]
    for idx, row in cluster_articles.iterrows():
        print(f"- {row['뉴스제목']}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


클러스터 결과:
                                     뉴스제목  cluster
0   배당 안주고 주가 누르더니 결국 상폐…헐값에 현금청산, 개미들 ..      0.0
1               "가족 먹여살리는 건 옛말"…날 위해 돈 쓴다      1.0
2     두산로보틱스와 합병하는 두산밥캣, 오는 11월 자사주 소각 방침      0.0
3   “콧대 높아서 사주고 싶지 않아”…생필품 주가 오를 때 ‘이 기..      0.0
4     강남·마용성 뛰는데 지방은 "손님 없어요"…서울 안에서도 양극화      0.0
..                                    ...      ...
95  "당국 압박 거세네" 은행 주담대 금리 줄인상..농협銀 0.2%..      1.0
96      김병환 "비트코인 현물 ETF 도입, 의견 분분…신중 의견"      0.0
97       와이지엔터, 걸그룹 '2NE1' 재결합 소식에 4%대 강세      0.0
98        한국 증시, 밸류업 위해 대수술하나…구조 재편론 '솔솔'      0.0
99         장현국 위메이드 부회장, 주식 155억원어치 전량 매도      0.0

[100 rows x 2 columns]

클러스터 0에 속하는 기사들:
- 배당 안주고 주가 누르더니 결국 상폐…헐값에 현금청산, 개미들 ..
- 두산로보틱스와 합병하는 두산밥캣, 오는 11월 자사주 소각 방침
- “콧대 높아서 사주고 싶지 않아”…생필품 주가 오를 때 ‘이 기..
- 강남·마용성 뛰는데 지방은 "손님 없어요"…서울 안에서도 양극화
- 언제 올랐지? 비트코인 '깜짝'…"두배 레버리지 사자" 베팅 늘..
- 두산로보틱스 공모가로 따져보니…합병시 ㈜두산 지분 42%→18%..
- "오르고 있었는데 너무하네"…뜻밖의 악재에 개미들 '눈물'
- [기획] 美대선 격랑… 세계경제 `시계 제로`
- 배당 안주고 주가 누르다 상폐 … 헐값에 현금청산 내몰린 개미
-

## K-means 군집화된 각 클러스터의 단어 빈도 분석

K-means 군집화 결과를 기반으로 각 클러스터에 대해 단어 빈도를 계산하고, 각 클러스터의 단어 빈도 배열을 출력

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


# 전처리된 텍스트를 사용하여 단어 빈도를 계산하는 함수
def calculate_word_frequencies(texts):
    # CountVectorizer 객체 생성
    vectorizer = CountVectorizer(
        tokenizer=lambda x: mecab.nouns(x),  # 형태소 분석기를 사용하여 명사만 추출
        min_df=2,                            # 최소 문서 빈도 설정
        max_df=0.80                          # 최대 문서 빈도 설정
    )
    # 단어 빈도 계산
    count_matrix = vectorizer.fit_transform(texts)

    # 단어 목록 및 빈도 배열 반환
    feature_names = vectorizer.get_feature_names_out()
    word_frequencies = count_matrix.toarray()

    return feature_names, word_frequencies

# 각 클러스터 그룹별로 CountVectorizer 적용
num_clusters = len(set(kmeans.labels_))

for cluster_num in range(num_clusters):
    print(f"group {cluster_num} countVectorizer data")

    # 해당 클러스터에 속하는 기사들 추출
    cluster_indices = [i for i, label in enumerate(kmeans.labels_) if label == cluster_num]
    cluster_texts = [preprocessed_texts[i] for i in cluster_indices]

    # 단어 빈도 계산
    feature_names, word_frequencies = calculate_word_frequencies(cluster_texts)

    # 단어 빈도 배열 출력
    print(word_frequencies)
    print("\n" + "="*80 + "\n")

group 0 countVectorizer data
[[4 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


group 1 countVectorizer data
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 0 0]]


group 2 countVectorizer data
[[0 0 0 ... 2 0 1]
 [0 0 2 ... 2 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 2 1 0]
 [0 0 0 ... 0 0 0]]




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## K-means 군집화된 클러스터에서 중복 기사 탐색

*   클러스터별 코사인 유사도를 이용한 중복 기사 검출
*   유사도가 0.9 이상인 기사를 중복으로 간주하고, 각 클러스터별로 중복 기사를 출력하고 이를 통해 중복된 기사들을 식별하고, 데이터 정제를 위한 추가 작업을 수행



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

checked = [[] for _ in range(num_clusters)]  # 중복 기사 인덱스를 저장할 리스트 초기화

cosine_sim = cosine_similarity(word_frequencies)  # 코사인 유사도 계산

# 중복 기사 제목을 저장할 리스트
duplicate_news = []

# 중복 기사 찾기
for i in range(len(cosine_sim)):
  for j in range(i + 1, len(cosine_sim)): # 코사인 유사도가 0.9 이상인 경우
    if 0.9 < cosine_sim[i, j] <= 0.999: # 유사도 범위에 따라 결과 달라짐.-> 문서 i와 j가 같은 클러스터에 있는지 확인
      if i not in checked[cluster_num]:
                checked[cluster_num].append(i)
      if j not in checked[cluster_num]:
                checked[cluster_num].append(j)

print("중복 기사들:")
for cluster_num in range(num_clusters):
    print(f"\n클러스터 {cluster_num}에서 중복 기사들:")
    # cluster_indices를 여기서 정의
    cluster_indices = [i for i, label in enumerate(kmeans.labels_) if label == cluster_num]
    if checked[cluster_num]:
        for idx in checked[cluster_num]:
            article_index = cluster_indices[idx]
            print(f"- {df.iloc[article_index]['뉴스제목']}")
            duplicate_news.append(df.iloc[article_index]['뉴스제목'])
    else:
        print("중복 기사가 없습니다")

중복 기사들:

클러스터 0에서 중복 기사들:
중복 기사가 없습니다

클러스터 1에서 중복 기사들:
중복 기사가 없습니다

클러스터 2에서 중복 기사들:
- "SK하이닉스 팔아라"…모건스탠리, 깜짝 전망 내놓은 이유
- 모건스탠리, SK하이닉스 빼고 아모레 담았다


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 중복 기사 목록

In [ ]:
# duplicate_news = [
#     "기사 제목 1",
#     "기사 제목 2",
#     "기사 제목 3",
#     ...
# ]

In [ ]:
duplicate_news

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['"SK하이닉스 팔아라"…모건스탠리, 깜짝 전망 내놓은 이유', '모건스탠리, SK하이닉스 빼고 아모레 담았다']

## 한국어 '오후'와 '오전'을 'PM'과 'AM'으로 변환 후 날짜와 시간 형식으로 변환


1.  한국어 '오후'와 '오전'을 'PM'과 'AM'으로 변환
*   이 단계에서는 문자열에서 '오후'를 'PM'으로, '오전'을 'AM'으로 변환합니다. 이는 pd.to_datetime 함수가 12시간제 형식의 시간 문자열을 이해할 수 있도록 돕기 위함

2.   날짜와 시간을 YYYY-MM-DD HH:MM:SS 형식으로 변환
*  pd.to_datetime 함수를 사용하여 문자열을 날짜와 시간으로 변환합니다. 여기서 %p는 AM/PM을 나타내고, %I는 12시간제를 나타



In [ ]:
# 한국어 '오후'와 '오전'을 'PM'과 'AM'으로 변환
def convert_korean_time_format(dt_str):
    # AM/PM 변환
    dt_str = re.sub(r'오후', 'PM', dt_str)
    dt_str = re.sub(r'오전', 'AM', dt_str)
    # 시간을 12시간제로 변환
    return dt_str

df['날짜'] = df['날짜'].apply(convert_korean_time_format)

# 날짜와 시간을 'YYYY-MM-DD HH:MM:SS' 형식으로 변환
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y.%m.%d. %p %I:%M')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 중복 기사 필터링 및 최신 기사 유지

1.   뉴스제목이 duplicate_news 리스트에 있는 기사를 필터링
2.   필터링된 기사를 날짜 기준으로 정렬하여 최신 기사가 위로 오도록 정렬함
3.   최신 기사를 제외한 중복 기사를 선택
4.   원본 데이터프레임에서 중복 기사를 삭제
5.  데이터프레임의 인덱스를 재정렬

In [ ]:
# 1. '뉴스제목' 열이 duplicate_news 리스트에 있는 기사만 필터링
filtered_df = df[df['뉴스제목'].isin(duplicate_news)]

# 2. 날짜 기준으로 정렬 (최신 기사가 맨 위로 오도록 정렬함)
sorted_filtered_df = filtered_df.sort_values(by='날짜', ascending=False)

# 3. 최신 기사를 제외한 나머지 기사를 선택
to_remove_df = sorted_filtered_df[1:]

# 4. 제거할 기사의 인덱스 추출
indices_to_remove = to_remove_df.index

# 5. 원본 데이터프레임에서 나머지 기사를 삭제
df = df.drop(index=indices_to_remove)

# 6. 인덱스 재정렬
df = df.reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 엑셀 파일로 저장
current_date = datetime.now().strftime("%Y%m%d")
file_name = f'duplicate_eliminated_{current_date}.xlsx'
df.to_excel(file_name, index=False)

df = pd.read_excel(file_name)
df